# Test the distilled Flan T5

In [1]:
import sys
sys.path.append('..')

In [2]:
import datasets
import torch
import re
import numpy as np

from tqdm import tqdm
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, MaxLengthCriteria
from src.data_utils import GSM8KCodexAugmentedDataset

/opt/conda/envs/llm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0,1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0,1


In [4]:
gsm8k = load_dataset('gsm8k', 'main')
gsm8k_test = gsm8k['test']

validation_index = np.load('../lib_prompt/validation_index.npy')
validation_data = gsm8k['train'].select(validation_index)

Found cached dataset gsm8k (/home/yaofu/.cache/huggingface/datasets/gsm8k/main/1.1.0/37bfb08b1d4fcbb01f06b03d9e1ef5f1fcbd4d3af3d08842c50d7305091285ba)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 331.33it/s]


In [7]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")

In [8]:
model = T5ForConditionalGeneration.from_pretrained("/mnt/data_10t/flan_t5_distill/checkpoints/0.0.2.0_epoch_0_iter_100", device_map='auto')

In [9]:
!nvidia-smi

Thu Dec 22 03:37:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:00:05.0 Off |                    0 |
| N/A   35C    P0    73W / 400W |  22175MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:00:06.0 Off |                    0 |
| N/A   

In [16]:
input_text = gsm8k['train'][1000]['question']

In [10]:
input_text = gsm8k['train'][0]['question']

In [11]:
prompt = open('../lib_prompt/prompt_simple_4_cases.txt').read()

In [17]:
prompt_q = prompt + '\nQuestion: ' + input_text + "\nLet's think step by step"

In [18]:
print(prompt_q)

Question: Ivan has a bird feeder in his yard that holds two cups of birdseed. Every week, he has to refill the emptied feeder. Each cup of birdseed can feed fourteen birds, but Ivan is constantly chasing away a hungry squirrel that steals half a cup of birdseed from the feeder every week. How many birds does Ivan’s bird feeder feed weekly?
Let's think step by step
The squirrel steals 1/2 cup of birdseed every week, so the birds eat 2 - 1/2 = 1 1/2 cups of birdseed.
Each cup feeds 14 birds, so Ivan’s bird feeder feeds 14 * 1 1/2 = 21 birds weekly.
The answer is 21

Question: Samuel took 30 minutes to finish his homework while Sarah took 1.3 hours to finish it. How many minutes faster did Samuel finish his homework than Sarah?
Let's think step by step
Since there are 60 minutes in 1 hour, then 1.3 hours is equal to 1.3 x 60 = 78 minutes.
Thus, Samuel is 78 – 30 = 48 minutes faster than Sarah.
The answer is 48

Question: Julia bought 3 packs of red balls, 10 packs of yellow balls, and 8 p

In [19]:
input_ids = tokenizer(prompt_q, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda:0")
input_ids.size()

torch.Size([1, 454])

In [44]:
input_ids

tensor([[11860,    10,    27,  2132,    65,     3,     9,  5963, 22916,    16,
           112,  6178,    24,  4532,   192, 12294,    13,  6331,  6958,     5,
          2181,   471,     6,     3,    88,    65,    12, 26210,     8,     3,
          9045,  5973, 22916,     5,  1698,  4119,    13,  6331,  6958,    54,
          3305, 27137,  6331,     6,    68,    27,  2132,    19,  4259,     3,
         22020,   550,     3,     9, 13802, 26364,    24, 11332,     7,   985,
             3,     9,  4119,    13,  6331,  6958,    45,     8, 22916,   334,
           471,     5,   571,   186,  6331,   405,    27,  2132,    22,     7,
          5963, 22916,  3305,  5547,    58,  1563,    31,     7,   317,  1147,
            57,  1147,    37, 26364, 11332,     7,  7739,  4119,    13,  6331,
          6958,   334,   471,     6,    78,     8,  6331,     3,  1544,   204,
             3,    18,  7739,  3274,   209,  7739, 12294,    13,  6331,  6958,
             5,  1698,  4119,  3305,     7,   968,  

In [23]:
outputs = model.generate(input_ids, max_length=256, sampling=True)
print(tokenizer.decode(outputs[0]))

ValueError: The following `model_kwargs` are not used by the model: ['sampling'] (note: typos in the generate arguments will also show up in this list)

In [29]:
dec_input = torch.tensor([[0]]).to('cuda:0')

In [32]:
outputs = model(input_ids=input_ids, decoder_input_ids=dec_input, return_dict=True)

In [35]:
outputs.logits[0, 0].topk(k=5)

torch.return_types.topk(
values=tensor([11.4741, 11.4652, 11.3593, 11.3318, 11.3148], device='cuda:0',
       grad_fn=<TopkBackward0>),
indices=tensor([32050,     0, 32026, 32006,  6385], device='cuda:0'))

In [ ]:
def silly_decode(model, batch, t):
    tgt_ids = batch['tgt_input_ids']
    for _ in range(t):
        out_dict_t = model(input_ids=processed_batch['src_input_ids'].to('cuda:0')
                           decoder_input_ids=
                          )
    return 